In [1]:
import pandas as pd

true_news = pd.read_csv('True.csv')
fake_news = pd.read_csv('Fake.csv')

In [2]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
true_news['valid'] = 1
fake_news['valid'] = 0

In [4]:
true_news = true_news.drop(['subject', 'date'], axis=1)
fake_news = fake_news.drop(['subject', 'date'], axis=1)

In [5]:
news_data = pd.concat([true_news, fake_news], ignore_index=True)
news_data = news_data.sample(frac=1)

In [6]:
import re
def remove_punctuations(text):
    return re.sub(r"[^\w\s]", "", text)
news_data[['title', 'text']] = news_data[['title', 'text']].apply(lambda x: x.str.lower())
news_data[['title', 'text']] = news_data[['title', 'text']].applymap(remove_punctuations)
news_data.head()

,title,text,valid
30667,philadelphia committee passes bill forcing sto...,what s more important protecting the dignity o...,0
1673,trump choice for russia ambassador no question...,washington reuters former utah governor jon h...,1
28983,president obama just did something amazing to...,if you ve ever raised a child while living in ...,0
36287,famous rhode island dancing cop fired for prot...,welcome to the new america where there can b...,0
32608,stunning first lady melania trump wears red to...,president donald trump and first lady melania ...,0


In [16]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
def remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens_without_sw = [token for token in tokens if not token.lower() in stopwords.words('english')]
    return " ".join(tokens_without_sw)

news_data[['title', 'text']] = news_data[['title', 'text']].applymap(remove_stopwords)
news_data.head()

[nltk_data] Downloading package stopwords to /home/yashwant-
[nltk_data]     bhosale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,text,valid
30667,philadelphia committee passes bill forcing sto...,important protecting dignity customers shop lo...,0
1673,trump choice russia ambassador question russia...,washington reuters former utah governor jon hu...,1
28983,president obama something amazing help poor co...,ever raised child living poverty know struggle...,0
36287,famous rhode island dancing cop fired protesti...,welcome new america opposition left serious co...,0
32608,stunning first lady melania trump wears red gr...,president donald trump first lady melania trum...,0


In [17]:
from sklearn.model_selection import train_test_split
news_data['text'] = news_data['title'] + ' ' + news_data['text']
news_data = news_data.drop(['title'], axis=1)
X = news_data['text']
Y = news_data['valid']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_transformed, y_train)
y_pred = model.predict(X_test_transformed)

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print(f"accuracy score: {accuracy_score(y_test, y_pred)}")
print(f"precision score: {precision_score(y_test, y_pred)}")
print(f"recall score: {recall_score(y_test, y_pred)}")
print(f"f1 score: {f1_score(y_test, y_pred)}")
print(f"confusion matrix: {confusion_matrix(y_test, y_pred)}")

accuracy score: 0.9887527839643653
precision score: 0.9857109393300538
recall score: 0.9905838041431262
f1 score: 0.9881413643301631
confusion matrix: [[4671   61]
 [  40 4208]]
